## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Husavik,IS,66.0449,-17.3389,35.82,83,99,14.90,overcast clouds
1,1,Tuktoyaktuk,CA,69.4541,-133.0374,-9.40,70,20,11.50,few clouds
2,2,Samarai,PG,-10.6167,150.6667,86.86,68,66,15.88,broken clouds
3,3,Cayenne,GF,4.9333,-52.3333,80.10,1,75,4.61,broken clouds
4,4,Bluff,NZ,-46.6000,168.3333,58.87,74,0,6.62,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Samarai,PG,-10.6167,150.6667,86.86,68,66,15.88,broken clouds
3,3,Cayenne,GF,4.9333,-52.3333,80.10,1,75,4.61,broken clouds
6,6,Saint-Philippe,RE,-21.3585,55.7679,78.39,81,20,10.51,few clouds
15,15,Rockhampton,AU,-23.3833,150.5000,82.29,61,48,9.22,scattered clouds
19,19,Puerto Ayora,EC,-0.7393,-90.3518,78.12,85,100,7.29,overcast clouds
21,21,Payo,ID,-3.7508,103.6372,88.99,56,16,3.33,few clouds
30,30,Atuona,PF,-9.8000,-139.0333,79.43,78,15,18.28,few clouds
31,31,Honiara,SB,-9.4333,159.9500,81.37,81,100,5.03,overcast clouds
35,35,Hervey Bay,AU,-25.2986,152.8535,76.46,76,19,7.87,few clouds
37,37,Vaini,TO,-21.2000,-175.2000,80.76,78,40,11.50,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                181
City                   181
Country                181
Lat                    181
Lng                    181
Max Temp               181
Humidity               181
Cloudiness             181
Wind Speed             181
Current Description    181
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.count()

City_ID                181
City                   181
Country                181
Lat                    181
Lng                    181
Max Temp               181
Humidity               181
Cloudiness             181
Wind Speed             181
Current Description    181
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Samarai,PG,86.86,broken clouds,-10.6167,150.6667,
3,Cayenne,GF,80.10,broken clouds,4.9333,-52.3333,
6,Saint-Philippe,RE,78.39,few clouds,-21.3585,55.7679,
15,Rockhampton,AU,82.29,scattered clouds,-23.3833,150.5000,
19,Puerto Ayora,EC,78.12,overcast clouds,-0.7393,-90.3518,
21,Payo,ID,88.99,few clouds,-3.7508,103.6372,
30,Atuona,PF,79.43,few clouds,-9.8000,-139.0333,
31,Honiara,SB,81.37,overcast clouds,-9.4333,159.9500,
35,Hervey Bay,AU,76.46,few clouds,-25.2986,152.8535,
37,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
#    print("get latitude") #debug
    lat = row["Lat"]
#    print("get longitude") #debug
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
#    print("params location step complete") #debug
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
#    print("set up base url step complete") #debug

#    print("make request and retrieve json step start") #debug
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
#    print("make request and retrieve json step complete") #debug    

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
#        print("entered try") #debug
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
#        print("hotel_df.loc step complete") #debug
    except (IndexError):
        print("Hotel not found... skipping.")

get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and re

make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up 

make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up 

make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
Hotel not found... skipping.
get latitude
get longitude
params location step complete
set u

make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
hotel_df.loc step complete
get latitude
get longitude
params location step complete
set up base url step complete
make request and retrieve json step start
make request and retrieve json step complete
entered try
Hotel not found... skipping.
get latitude
get longitude
params location step complete
set u

In [24]:
# check hotel_df
hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
654,Sao Joao Da Barra,BR,77.18,broken clouds,-21.6403,-41.0511,Pousada Porto De Canoas
658,Wewak,PG,79.50,overcast clouds,-3.5534,143.6268,In Wewak Boutique Hotel
679,Gamba,GA,77.43,light rain,-2.6500,10.0000,
683,Pali,IN,89.42,clear sky,25.7667,73.3333,HOTEL THE AMBA VILAS
687,Carutapera,BR,75.65,light rain,-1.1950,-46.0200,Lidera
690,Townsville,AU,88.68,scattered clouds,-19.2500,146.8000,Rydges Southbank Townsville
692,Ocos,GT,81.57,broken clouds,14.5094,-92.1933,Auto Hotel El Soñador
693,Micheweni,TZ,81.32,broken clouds,-4.9667,39.8333,
694,Saint-Paul,RE,79.74,clear sky,-21.0096,55.2707,Boucan Canot
701,Khawhai,IN,75.87,broken clouds,23.3667,93.1167,Homestay Khawhai


In [10]:
hotel_df.count()

City                   181
Country                181
Max Temp               181
Current Description    181
Lat                    181
Lng                    181
Hotel Name             181
dtype: int64

In [28]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df = clean_hotel_df[(clean_hotel_df["Hotel Name"] != "")]
clean_hotel_df.count()

City                   169
Country                169
Max Temp               169
Current Description    169
Lat                    169
Lng                    169
Hotel Name             169
dtype: int64

In [29]:
clean_hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
652,Yulara,AU,89.08,clear sky,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
653,Turbat,PK,84.15,clear sky,26.0023,63.0440,Mubashar Quarters
654,Sao Joao Da Barra,BR,77.18,broken clouds,-21.6403,-41.0511,Pousada Porto De Canoas
658,Wewak,PG,79.50,overcast clouds,-3.5534,143.6268,In Wewak Boutique Hotel
683,Pali,IN,89.42,clear sky,25.7667,73.3333,HOTEL THE AMBA VILAS
687,Carutapera,BR,75.65,light rain,-1.1950,-46.0200,Lidera
690,Townsville,AU,88.68,scattered clouds,-19.2500,146.8000,Rydges Southbank Townsville
692,Ocos,GT,81.57,broken clouds,14.5094,-92.1933,Auto Hotel El Soñador
694,Saint-Paul,RE,79.74,clear sky,-21.0096,55.2707,Boucan Canot
701,Khawhai,IN,75.87,broken clouds,23.3667,93.1167,Homestay Khawhai


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temperature</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#              max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))